In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

#other libraries
# from tqdm import tqdm
# import time
# import random
import os
import sys
from pathlib import Path
import h5py

#torch specific
import torch
import torchvision as torchv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils import data

from sklearn.model_selection import train_test_split
import wandb

from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn

In [2]:
module_path = str(Path.cwd().parents[0].parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
# from plotCreator import *
import trainers

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "bh" / "BH_n4_M10_res50_15000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "sph" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_15000_events.h5")

Running on the GPU


In [3]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

In [4]:
print(bhArray.shape)
print(sphArray.shape)

(15000, 50, 50, 3)
(15000, 50, 50, 3)


In [5]:
dataArray = np.concatenate((bhArray,sphArray),axis=0)
labelsArray = np.concatenate((np.zeros(np.shape(bhArray)[0]),np.ones(np.shape(sphArray)[0])),axis=0)

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [7]:
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [8]:
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [9]:
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [10]:
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))
        
        x = self.relu(self.batch_norm2(self.conv2(x)))
        
        x = self.conv3(x)
        x = self.batch_norm3(x)
        
        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)
        
        return x

In [11]:
class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)
        
    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )
            
        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        
        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))
            
        return nn.Sequential(*layers)

In [12]:
from torchsummary import summary

resnet101 = ResNet(Bottleneck, [3, 4, 23, 3], 2, 3).to(device)
summary(resnet101, (3, 50, 50))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 25, 25]           9,408
       BatchNorm2d-2           [-1, 64, 25, 25]             128
              ReLU-3           [-1, 64, 25, 25]               0
         MaxPool2d-4           [-1, 64, 13, 13]               0
            Conv2d-5           [-1, 64, 13, 13]           4,160
       BatchNorm2d-6           [-1, 64, 13, 13]             128
              ReLU-7           [-1, 64, 13, 13]               0
            Conv2d-8           [-1, 64, 13, 13]          36,928
       BatchNorm2d-9           [-1, 64, 13, 13]             128
             ReLU-10           [-1, 64, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]          16,640
      BatchNorm2d-12          [-1, 256, 13, 13]             512
           Conv2d-13          [-1, 256, 13, 13]          16,640
      BatchNorm2d-14          [-1, 256,

In [13]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'Test accuracy'
        },
    'parameters': {
        'loss': {'values': ['customLoss', 'hinge', 'cross']},
        'epoch': {'values': [10]},
        'batch_size': {'values': [50]},
        'gamma': {'values': [1]},
        'learning_rate': {'values': [0.01]},
        'optimizer': {'values': ['adam']},
     }
}
sweep_id = wandb.sweep(sweep=sweep_configuration, project="ResNet101_Sweep")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 6o6qi6ju
Sweep URL: https://wandb.ai/g13hvl2023/ResNet101_Sweep/sweeps/6o6qi6ju


In [14]:
#wrapper
def trainFunction():
    with wandb.init(project="ResNet101_Sweep", name="ResNet101_Sweep"):
        config = wandb.config
        model = ResNet(Bottleneck, [3, 4, 23, 3], 2, 3).to(device)
        trainers.sweep(model, train, test, config)

In [15]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project="ResNet101_Sweep")
wandb.agent(sweep_id, function=trainFunction, count=10)

Create sweep with ID: 9k3uqwgn
Sweep URL: https://wandb.ai/g13hvl2023/ResNet101_Sweep/sweeps/9k3uqwgn


wandb: Agent Starting Run: 2q9hfs1u with config:
wandb: 	batch_size: 50
wandb: 	epoch: 20
wandb: 	gamma: 1
wandb: 	learning_rate: 0.01
wandb: 	loss: customLoss
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 591291 (g13hvl2023). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


BH accuracy,▁▆▆▇▇███▇█▇█▇▇█▇███▇█
SPH accuracy,▇▆██▇▆▁▆█▇▇▇▇█▇▇█▇▇█▇
Test accuracy,▁▆▆▇▇█▇██████████████
Test epoch_loss,█▄▄▂▃▂▃▁▁▁▂▁▁▁▁▂▁▁▁▁▁
Test precision,▇▆██▇▆▁▆█▇▇▇▇█▇▇█▇▇█▇
Test recall,▁▆▅▆▆███▇▇▇█▇▇▇▇▇█▇▇█
Train accuracy,▁▄▅▅▆▇▇▇█▅▇▇▇▇▇██▆▇█
Train epoch_loss,█▆▅▄▃▂▂▂▁▄▂▃▂▂▂▁▁▄▂▁
BH accuracy,0.88024
SPH accuracy,0.94796
Test accuracy,0.91373


wandb: Agent Starting Run: psutqiu9 with config:
wandb: 	batch_size: 50
wandb: 	epoch: 20
wandb: 	gamma: 1
wandb: 	learning_rate: 0.01
wandb: 	loss: customLoss
wandb: 	optimizer: sdg
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


c:\Users\student\Anaconda3\envs\DAT191-gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
